<a href="https://colab.research.google.com/github/ildimas/NeuralNetworksInManagment/blob/main/%D0%98%D0%BB%D1%8C%D1%8E%D1%89%D0%B5%D0%BD%D1%8F_%D0%9D%D0%A2%D0%92%D0%A3_3_%D0%97%D0%B0%D0%B4%D0%B0%D0%BD%D0%B8%D0%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание: LLM и VLM

## Часть 1: Языковые модели на основе трансформеров

### 1.1 Fine-tuning модели на специфическом корпусе

In [ ]:
pip install transformers datasets torch accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 5.9 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

#### Выбор модели
Для нашего задания мы будем использовать DistilGPT2, так как она:
1. Легкая и быстрая в обучении
2. Хорошо работает на ограниченных ресурсах
3. Имеет хорошую документацию и поддержку

In [ ]:
model_name = "distilgpt2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

#### Подготовка данных
Для fine-tuning мы будем использовать датасет научных абстрактов

In [ ]:
dataset = load_dataset("scientific_papers", "arxiv", split="train[:1000]")

def preprocess_function(examples):
    return tokenizer(examples["abstract"], truncation=True, max_length=512)

tokenized_dataset = dataset.map(preprocess_function, batched=True)

#### Настройка параметров обучения

In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    fp16=True,
    save_steps=100,
    logging_steps=10,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=100,
    evaluation_strategy="steps",
    eval_steps=100,
    load_best_model_at_end=True,
)

### 2.2 Условная генерация текста

In [ ]:
def generate_text(prompt, strategy="greedy", **kwargs):
    inputs = tokenizer(prompt, return_tensors="pt")

    if strategy == "greedy":
        outputs = model.generate(
            inputs["input_ids"],
            max_length=100,
            num_return_sequences=1,
            **kwargs
        )
    elif strategy == "beam":
        outputs = model.generate(
            inputs["input_ids"],
            max_length=100,
            num_beams=5,
            num_return_sequences=1,
            **kwargs
        )
    elif strategy == "top_k":
        outputs = model.generate(
            inputs["input_ids"],
            max_length=100,
            top_k=50,
            num_return_sequences=1,
            **kwargs
        )
    elif strategy == "nucleus":
        outputs = model.generate(
            inputs["input_ids"],
            max_length=100,
            top_p=0.92,
            num_return_sequences=1,
            **kwargs
        )

    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Часть 3: Работа с малыми и средними LLM

## 3.1 Исследование возможностей LLM

В этой части мы будем работать с моделью Phi-2, которая является хорошим выбором для работы в Colab благодаря:
1. Компактному размеру (2.7B параметров)
2. Хорошей производительности
3. Возможности эффективной квантизации

In [ ]:
pip install transformers torch accelerate bitsandbytes peft

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd
import numpy as np
from peft import LoraConfig, get_peft_model
import json
from tqdm import tqdm

### Загрузка модели и токенизатора

In [ ]:
model_name = "microsoft/phi-2"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    trust_remote_code=True
)

### 3.1.1 Реализация различных стратегий промптов

In [ ]:
def generate_response(prompt, temperature=0.7, max_length=200):
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    outputs = model.generate(
        **inputs,
        max_length=max_length,
        temperature=temperature,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

zero_shot_prompt = """Классифицируй следующий текст как позитивный, негативный или нейтральный:
Текст: {text}
Классификация:"""

few_shot_prompt = """Классифицируй следующий текст как позитивный, негативный или нейтральный.

Примеры:
Текст: Я очень доволен покупкой, товар превзошел все ожидания!
Классификация: позитивный

Текст: Ужасный сервис, никогда больше не вернусь в этот магазин.
Классификация: негативный

Текст: Купил обычную ручку в магазине.
Классификация: нейтральный

Текст: {text}
Классификация:"""

cot_prompt = """Реши следующую задачу пошагово:

Задача: {task}

Давай решим это пошагово:
1)"""

### 3.1.2 Сравнение различных стратегий промптов

In [ ]:
test_texts = [
    "Новый фильм оказался просто потрясающим!",
    "Сервис в ресторане был ужасно медленным.",
    "Сегодня обычный рабочий день."
]

results = []
for text in test_texts:
    zero_shot_result = generate_response(zero_shot_prompt.format(text=text))
    few_shot_result = generate_response(few_shot_prompt.format(text=text))

    results.append({
        "text": text,
        "zero_shot": zero_shot_result,
        "few_shot": few_shot_result
    })

pd.DataFrame(results)

### 3.1.3 Исследование влияния температуры

In [ ]:
def compare_temperatures(prompt, temperatures=[0.3, 0.7, 1.0]):
    results = {}
    for temp in temperatures:
        results[f"temp_{temp}"] = generate_response(prompt, temperature=temp)
    return results

test_prompt = "Напиши короткое стихотворение о весне."
temperature_results = compare_temperatures(test_prompt)

for temp, result in temperature_results.items():
    print(f"\n=== {temp} ===\n{result}")

## 3.2 Адаптация LLM для специфической задачи

В этой части мы реализуем LoRA для адаптации модели под конкретную задачу

In [ ]:
lora_config = LoraConfig(
    r=16,  # ранг адаптации
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

NameError: name 'LoraConfig' is not defined

### Подготовка данных для дообучения

In [ ]:
medical_data = [
    {"question": "Какие симптомы у гриппа?", "answer": "Основные симптомы гриппа включают высокую температуру, кашель, боль в горле, насморк, мышечные боли и общую слабость."},
    {"question": "Как лечить простуду?", "answer": "При простуде рекомендуется отдых, обильное питье, прием жаропонижающих при высокой температуре и симптоматическое лечение."}
]

def prepare_training_data(data):
    formatted_data = []
    for item in data:
        prompt = f"Вопрос: {item['question']}\nОтвет: {item['answer']}"
        formatted_data.append(prompt)
    return formatted_data

training_data = prepare_training_data(medical_data)

### Настройка параметров обучения

In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=10,
    save_strategy="epoch",
    evaluation_strategy="epoch",
    load_best_model_at_end=True,
)

### Сравнение производительности до и после адаптации

In [ ]:
def evaluate_model(question):
    prompt = f"Вопрос: {question}\nОтвет:"
    return generate_response(prompt)

test_questions = [
    "Какие основные симптомы COVID-19?",
    "Как правильно измерять давление?"
]

for question in test_questions:
    print(f"\nВопрос: {question}")
    print(f"Ответ: {evaluate_model(question)}")

## Часть 4: Мультимодальные трансформеры и VLM

### 4.1 Анализ визуально-языковых представлений

В этой части мы будем работать с моделью CLIP ViT-B/32, которая позволяет:
1. Создавать совместное пространство признаков для изображений и текста
2. Выполнять поиск изображений по текстовому запросу
3. Генерировать текстовые описания изображений

In [ ]:
pip install torchvision pillow ftfy regex tqdm
pip install git+https://github.com/openai/CLIP.git

In [ ]:
import torch
import clip
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from torchvision.transforms import Compose, Resize, CenterCrop, ToTensor, Normalize
from sklearn.manifold import TSNE
import requests
from io import BytesIO

### Загрузка модели CLIP

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model, preprocess = clip.load("ViT-B/32", device=device)

def load_image(image_path):
    if image_path.startswith('http'):
        response = requests.get(image_path)
        image = Image.open(BytesIO(response.content))
    else:
        image = Image.open(image_path)
    return image

### 4.1.1 Исследование совместного пространства признаков

In [ ]:
def get_image_features(image_path):
    image = load_image(image_path)
    image_input = preprocess(image).unsqueeze(0).to(device)
    with torch.no_grad():
        image_features = model.encode_image(image_input)
    return image_features.cpu().numpy()

def get_text_features(text):
    text_input = clip.tokenize([text]).to(device)
    with torch.no_grad():
        text_features = model.encode_text(text_input)
    return text_features.cpu().numpy()

def compute_similarity(image_features, text_features):
    similarity = np.dot(image_features, text_features.T)
    return similarity[0][0]

### 4.1.2 Визуализация сходства между изображениями и текстом

In [ ]:
def visualize_similarities(image_paths, text_queries):
    image_features = np.vstack([get_image_features(path) for path in image_paths])
    text_features = np.vstack([get_text_features(query) for query in text_queries])

    similarity_matrix = np.dot(image_features, text_features.T)

    plt.figure(figsize=(10, 8))
    plt.imshow(similarity_matrix, cmap='viridis')
    plt.colorbar()
    plt.xticks(range(len(text_queries)), text_queries, rotation=45)
    plt.yticks(range(len(image_paths)), [f'Image {i+1}' for i in range(len(image_paths))])
    plt.title('Матрица сходства между изображениями и текстовыми запросами')
    plt.tight_layout()
    plt.show()

### 4.2 Адаптация VLM для специфической задачи

В этой части мы адаптируем модель для задачи генерации описаний изображений

In [ ]:
def generate_image_description(image_path, candidate_descriptions):
    image_features = get_image_features(image_path)

    text_features = np.vstack([get_text_features(desc) for desc in candidate_descriptions])

    similarities = [compute_similarity(image_features, text_feat) for text_feat in text_features]

    best_idx = np.argmax(similarities)
    return candidate_descriptions[best_idx], similarities[best_idx]

### Тестирование генерации описаний

In [ ]:
test_image = "path_to_test_image.jpg"
candidate_descriptions = [
    "Красивый закат над морем",
    "Городской пейзаж в дождливый день",
    "Лесная тропинка в солнечный день",
    "Горный пейзаж с заснеженными вершинами"
]

description, confidence = generate_image_description(test_image, candidate_descriptions)
print(f"Лучшее описание: {description}")
print(f"Уверенность: {confidence:.2f}")

### Визуализация результатов

In [ ]:
def visualize_results(image_path, descriptions, similarities):
    image = load_image(image_path)
    plt.figure(figsize=(12, 6))

    plt.subplot(1, 2, 1)
    plt.imshow(image)
    plt.axis('off')
    plt.title('Исходное изображение')

    plt.subplot(1, 2, 2)
    y_pos = np.arange(len(descriptions))
    plt.barh(y_pos, similarities)
    plt.yticks(y_pos, descriptions)
    plt.xlabel('Сходство')
    plt.title('Сходство с различными описаниями')

    plt.tight_layout()
    plt.show()